In [1]:
# Instalación de librerías requeridas
%pip install sqlalchemy
%pip install psycopg2
%pip install psycopg2-binary
%pip install scikit-learn
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import yaml
from sqlalchemy import create_engine


In [4]:
with open('../../configBD/config.yml', 'r') as f:
    cfg        = yaml.safe_load(f)
    cfg_etl    = cfg['bodega']
    cfg_bd     = cfg['mensajeria']
cfg_etl       # verificación rápida

{'driver': 'postgresql+psycopg2',
 'host': 'proyectobodega.postgres.database.azure.com',
 'port': 5432,
 'user': 'adminbodega',
 'password': 'Goddess9039#',
 'db': 'proyectobodega'}

In [5]:
url_bd  = f"{cfg_bd['driver']}://{cfg_bd['user']}:{cfg_bd['password']}@{cfg_bd['host']}:{cfg_bd['port']}/{cfg_bd['db']}"
url_etl = f"{cfg_etl['driver']}://{cfg_etl['user']}:{cfg_etl['password']}@{cfg_etl['host']}:{cfg_etl['port']}/{cfg_etl['db']}"


In [6]:
cliente_bd  = create_engine(url_bd)     # base operativa
cliente_etl = create_engine(url_etl)    # Data Warehouse


In [7]:
dim_cliente   = pd.read_sql_table('dim_cliente',   url_etl)
dim_mensajero = pd.read_sql_table('dim_mensajero', url_etl)
dim_sede      = pd.read_sql_table('dim_sede',      url_etl)
dim_tiempo    = pd.read_sql_table('dim_tiempo',    url_etl)

# normaliza dim_tiempo → genera columna date
dim_tiempo = dim_tiempo.rename(columns={'Año':'year','Mes':'month','Dia':'day'})
dim_tiempo['date'] = pd.to_datetime(dim_tiempo[['year','month','day']]).dt.date


In [8]:
servicios      = pd.read_sql_table('mensajeria_servicio',        url_bd)
origenes       = pd.read_sql_table('mensajeria_origenservicio',  url_bd)
destinos       = pd.read_sql_table('mensajeria_destinoservicio', url_bd)
estados_srv    = pd.read_sql_table('mensajeria_estadosservicio', url_bd)
cat_estado     = pd.read_sql_table('mensajeria_estado',          url_bd)
cat_tipo_srv   = pd.read_sql_table('mensajeria_tiposervicio',    url_bd)


In [27]:
# ----- 0) copia base -----
fact_serv = servicios.copy()

# ----- 1) TiempoKey (fecha_solicitud) -----
fact_serv['FechaSolicitud']   = pd.to_datetime(fact_serv['fecha_solicitud']).dt.date
dim_tiempo_subset = dim_tiempo[['tiempo_key','date']].rename(columns={'tiempo_key':'TiempoKey'})
fact_serv = fact_serv.merge(dim_tiempo_subset,
                            left_on='FechaSolicitud',
                            right_on='date', how='left') \
                     .drop(columns=['date'])
print(fact_serv.columns.tolist())


# ----- 2) FK Cliente -----
fact_serv = fact_serv.merge(dim_cliente[['ClienteKey','cliente_id']],
                            on='cliente_id', how='left')
print(fact_serv.columns.tolist())


# ----- 3) FK Mensajero (titular) -----
fact_serv = fact_serv.merge(dim_mensajero[['MensajeroKey','user_id']],
                            left_on='mensajero_id', right_on='user_id',
                            how='left').drop(columns=['user_id'])
print(fact_serv.columns.tolist())


# ----- 4) FK Sede Origen y Destino -----
# 4a) unir origen
fact_serv = (
    fact_serv
    .merge(
        origenes[['id','cliente_id','ciudad_id']],
        left_on='origen_id', right_on='id',
        how='left',
        suffixes=('','_ori')      # id→id_ori
    )
    .rename(columns={
        'ciudad_id':      'ciudad_ori',
        'cliente_id_ori': 'cliente_ori'
    })
    .drop(columns=['id_ori'])    # <— borramos sólo id_ori, NO id
)

# 4b) unir destino
fact_serv = (
    fact_serv
    .merge(
        destinos[['id','cliente_id','ciudad_id']],
        left_on='destino_id', right_on='id',
        how='left',
        suffixes=('','_des')      # id→id_des
    )
    .rename(columns={
        'ciudad_id':      'ciudad_des',
        'cliente_id_des': 'cliente_des'
    })
    .drop(columns=['id_des'])    # <— borramos sólo id_des
)


# 4c  lookup en dim_sede
# fact_serv = fact_serv.merge(dim_sede[['SedeKey','ciudad_id','cliente_id']],
#                             left_on=['ciudad_ori','cliente_ori'],
#                             right_on=['ciudad_id','cliente_id'],
#                             how='left').rename(columns={'SedeKey':'SedeOrigenKey'}) \
#                      .drop(columns=['ciudad_id','cliente_id'])
# 4c) lookup en dim_sede para SedeOrigenKey
fact_serv = (
    fact_serv
    .merge(
        dim_sede[['SedeKey','ciudad_id','cliente_id']],
        left_on=['ciudad_ori','cliente_ori'],
        right_on=['ciudad_id','cliente_id'],
        how='left',
        suffixes=('','_dim')           # sufijo solo en cliente_id de dim_sede
    )
    .rename(columns={'SedeKey':'SedeOrigenKey'})
    .drop(columns=['ciudad_id','cliente_id_dim'])  # elimina la ciudad_id traída y el cliente_id_dim
)

# 4d) lookup en dim_sede para SedeDestinoKey (idéntico, cambiando sólo nombres)
fact_serv = (
    fact_serv
    .merge(
        dim_sede[['SedeKey','ciudad_id','cliente_id']],
        left_on=['ciudad_des','cliente_des'],
        right_on=['ciudad_id','cliente_id'],
        how='left',
        suffixes=('','_dim')
    )
    .rename(columns={'SedeKey':'SedeDestinoKey'})
    .drop(columns=['ciudad_id','cliente_id_dim'])
)

# ----- 5) TipoServicio -----
fact_serv = (
    fact_serv
    .merge(
        cat_tipo_srv[['id','nombre']],
        left_on='tipo_servicio_id',
        right_on='id',
        how='left',
        suffixes=('','_tipo')
    )
    .rename(columns={'nombre':'TipoServicio'})
    .drop(columns=['id_tipo'])
)

# ----- 6) EstadoServicio (último estado registrado) -----
# 6a) fusionar nombre del estado
# 6a) fusionar nombre del estado
estados_srv = (
    estados_srv
    .merge(
        cat_estado[['id','nombre']],
        left_on='estado_id',
        right_on='id',
        how='left',
        suffixes=('','_cat')
    )
    .rename(columns={'nombre':'EstadoNom'})
    .drop(columns=['id_cat'])
)

# ———————> dedupe columns <———————
estados_srv = estados_srv.loc[:, ~estados_srv.columns.duplicated()]


# 6b) timestamp con microsegundos
estados_srv['timestamp'] = pd.to_datetime(
    estados_srv['fecha'].astype(str) + ' ' + estados_srv['hora'].astype(str),
    format='%Y-%m-%d %H:%M:%S.%f',
    errors='coerce'
)

# 6c) extraer último estado
ult_estado = (
    estados_srv
    .sort_values('timestamp')
    .groupby('servicio_id', as_index=False)
    .tail(1)[['servicio_id','EstadoNom','timestamp']]
)

# 6d) merge en fact_serv
fact_serv = (
    fact_serv
    .merge(ult_estado,
           left_on='id', right_on='servicio_id',
           how='left')
    .rename(columns={'EstadoNom':'EstadoServicio'})
    .drop(columns=['servicio_id'])
)

# ————————————— CORRECCIÓN —————————————
# Elimina cualquier columna duplicada (por si quedaron dos 'EstadoServicio')
fact_serv = fact_serv.loc[:, ~fact_serv.columns.duplicated()]

# ----- 7) EsFinal (TRUE si el estado es uno de los finales) -----
finales = ['ENTREGADO','CERRADO','CANCELADO']
fact_serv['EsFinal'] = fact_serv['EstadoServicio'].isin(finales)

# ----- 8) Timestamps de tracking -----
fact_serv['Tiempo_Inicio'] = pd.to_datetime(
    fact_serv['fecha_solicitud'].astype(str) + ' ' + fact_serv['hora_solicitud'].astype(str),
    errors='coerce'
)
fact_serv['Tiempo_Mensajero_Asignado'] = pd.to_datetime(
    fact_serv['hora_visto_por_mensajero'], errors='coerce'
)

def _extrae_evt(df, estado_nombre, new_col):
    tmp = df[df['EstadoNom'] == estado_nombre].copy()
    tmp['ts'] = pd.to_datetime(
        tmp['fecha'].astype(str) + ' ' + tmp['hora'].astype(str),
        errors='coerce'
    )
    return (
        tmp.sort_values('ts')
           .groupby('servicio_id', as_index=False)
           .head(1)[['servicio_id','ts']]
           .rename(columns={'ts': new_col})
    )

for estado, new_col in [
    ('RECOGIDO_ORIGEN',     'Tiempo_Recogido_Origen'),
    ('ENTREGADO_DESTINO',   'Tiempo_Entregado_Destino'),
    ('CERRADO',             'Tiempo_Cerrado')
]:
    evt = _extrae_evt(estados_srv, estado, new_col)
    fact_serv = (
        fact_serv
        .merge(evt,
               left_on='id',
               right_on='servicio_id',
               how='left')
        .drop(columns=['servicio_id'])
    )


# ----- 9) limpieza de columnas intermedias -----
drop_cols = (
    [c for c in fact_serv.columns if c.startswith(('fecha_','hora_','ciudad_','cliente_'))]
    + ['servicio_id']
)
fact_serv = fact_serv.drop(columns=drop_cols, errors='ignore')


['id', 'descripcion', 'nombre_solicitante', 'fecha_solicitud', 'hora_solicitud', 'fecha_deseada', 'hora_deseada', 'nombre_recibe', 'telefono_recibe', 'descripcion_pago', 'ida_y_regreso', 'activo', 'novedades', 'cliente_id', 'destino_id', 'mensajero_id', 'origen_id', 'tipo_pago_id', 'tipo_servicio_id', 'tipo_vehiculo_id', 'usuario_id', 'prioridad', 'ciudad_destino_id', 'ciudad_origen_id', 'hora_visto_por_mensajero', 'visto_por_mensajero', 'descripcion_multiples_origenes', 'mensajero2_id', 'mensajero3_id', 'multiples_origenes', 'asignar_mensajero', 'es_prueba', 'descripcion_cancelado', 'FechaSolicitud', 'TiempoKey']
['id', 'descripcion', 'nombre_solicitante', 'fecha_solicitud', 'hora_solicitud', 'fecha_deseada', 'hora_deseada', 'nombre_recibe', 'telefono_recibe', 'descripcion_pago', 'ida_y_regreso', 'activo', 'novedades', 'cliente_id', 'destino_id', 'mensajero_id', 'origen_id', 'tipo_pago_id', 'tipo_servicio_id', 'tipo_vehiculo_id', 'usuario_id', 'prioridad', 'ciudad_destino_id', 'ciudad

In [30]:

# # 0) “Subconjunto” de las primeras 1 000 filas
# fact_serv_sub = fact_serv.head(1000)      # o fact_serv.iloc[:1000]
# print(fact_serv_sub.shape)                # debe mostrar (1000, n_columnas)
# print(fact_serv_sub.head())               # vistazo a las primeras filas

# # 1a) Limpia la transacción pendiente
# raw = cliente_etl.raw_connection()
# raw.rollback()
# raw.close()

# # 1b) Vuelca solo esas 1 000 filas con el mismo método
# fact_serv_sub.to_sql(
#     'fact_servicios',
#     cliente_etl,
#     if_exists='replace',
#     index=False,
#     method='multi',
#     chunksize=1000
# )

# 0) renombra el PK para que sea ServicioKey
fact_serv = fact_serv.rename(columns={'id':'ServicioKey'})

# 1) define el orden definitivo
cols_final = [
    'ServicioKey','TiempoKey','ClienteKey','SedeOrigenKey','SedeDestinoKey',
    'MensajeroKey','TipoServicio','EstadoServicio','EsFinal',
    'Tiempo_Inicio','Tiempo_Mensajero_Asignado',
    'Tiempo_Recogido_Origen','Tiempo_Entregado_Destino','Tiempo_Cerrado'
]

# 2) filtra solo esas columnas
fact_serv = fact_serv[cols_final]

# 3) muestra las primeras filas
print(fact_serv.head())




   ServicioKey  TiempoKey  ClienteKey  SedeOrigenKey  SedeDestinoKey  \
0           34     7152.0           7           16.0            16.0   
1           34     7152.0           7           16.0            17.0   
2           34     7152.0           7           16.0            19.0   
3           34     7152.0           7           16.0            24.0   
4           34     7152.0           7           16.0            49.0   

   MensajeroKey TipoServicio EstadoServicio  EsFinal       Tiempo_Inicio  \
0           NaN      Clínico       Iniciado    False 2023-10-26 09:46:03   
1           NaN      Clínico       Iniciado    False 2023-10-26 09:46:03   
2           NaN      Clínico       Iniciado    False 2023-10-26 09:46:03   
3           NaN      Clínico       Iniciado    False 2023-10-26 09:46:03   
4           NaN      Clínico       Iniciado    False 2023-10-26 09:46:03   

  Tiempo_Mensajero_Asignado Tiempo_Recogido_Origen Tiempo_Entregado_Destino  \
0                       NaT    